In [1]:
import numpy as np
from collections import defaultdict
import random

In [2]:
class Cluster:
    #centroid: tâm của cụm
    #members: danh sách dữ liệu trong cụm
    def __init__(self):
        self._centroid= None
        self._members= []
    def reset_members(self):
        self._members= []
    def add_member(self, member):
        self._members.append(member)

In [3]:
class Member:
    """
    r_d: biểu diễn tf_idf của văn bản d
    label: newsgroup của văn bản d
    doc_id: tên file chứa văn bản d
    """
    def __init__(self, r_d, label= None, doc_id= None):
    #Khởi tạo
        self._r_d= r_d
        self._label= label
        self._doc_id= doc_id

In [4]:
class Kmeans:
    """
    _num_cluster: số cluster đang xét
    _cluster: list các cluster
    _E : list các centroid
    _S : list similarity(_r_d, _E_k)
    """
    #Khởi tạo
    def __init__(self, num_clusters):
        self._num_clusters= num_clusters
        self._cluster= [Cluster() for _ in range(self._num_clusters)]
        self._E= [] # List các centroid
        self._S= 0
    
    def load_data(self, data_path):
        # Đọc dữ liệu từ file
        def sparse_to_dense(sparse_r_d, vocab_size):
            r_d= [0.0 for _ in range(vocab_size)]
            indices_tfidfs = sparse_r_d.split()
            #tách và lấy dữ liệu giữa dấu :
            for index_tfidf in indices_tfidfs:
                index = int(index_tfidf.split(':')[0])
                tfidf = float(index_tfidf.split(':')[1])
                r_d[index] = tfidf
            return np.array(r_d)
        with open (data_path) as f:
            d_lines= f.read().splitlines()
        
        with open('./datasets/20news-bydate/words_idfs.txt') as f:
            vocab_size= len(f.read().splitlines())
        
        self._data=[]
        self._label_count = defaultdict(int)
        for data_id, d in enumerate(d_lines):
            features= d.split('<fff>') #lấy từng dữ liệu từ file
            label, doc_id = int(features[0]), int(features[1])
            self._label_count[label] += 1
            
            r_d = sparse_to_dense(sparse_r_d= features[2], vocab_size= vocab_size)
            self._data.append(Member(r_d = r_d, label= label, doc_id= doc_id))
            
    def random_init(self, seed_value):
    #khởi tạo random
        d = 0
        for cluster in self._cluster:
            d += 1
            np.random.seed(int(seed_value) + d)
            temp = np.random.randint(len(self._data))
            cluster._centroid = self._data[temp]._r_d
            
    def compute_similarity(self, member, centroid):
    #đánh giá similarity
        mem_r_d = np.array(member._r_d)
        cen_r_d = np.array(centroid)
        dis = np.linalg.norm(mem_r_d - cen_r_d) + 1e-12
        return 1./dis
    
    def select_cluster_for(self, member):
    #xác định cụm cho từng điểm dữ liệu (chọn cluster cho từng member theo centroid đã biết)
        best_fit_cluster = None
        max_similarity = -1
        for cluster in self._cluster:
            similarity= self.compute_similarity(member, cluster._centroid)
            if (similarity > max_similarity):
                max_similarity= similarity
                best_fit_cluster= cluster
        best_fit_cluster.add_member(member)
        return max_similarity
    
    def update_centroid_of(self, cluster):
    #xác định lại tâm (cố định member của từng cụm, tính toán lại cluster)
        member_r_ds= [member._r_d for member in cluster._members]
        average_r_d= np.mean(member_r_ds, axis=0)
        sqrt_sum_sqr = np.sqrt(np.sum(average_r_d ** 2))
        new_centroid = np.array([value / sqrt_sum_sqr for value in average_r_d])
        
        cluster._centroid = new_centroid
    
    def stopping_condition(self, criterion, threshold):
    # Điều kiện dừng với các tham số là điều kiện và ngưỡng dừng
        criteria = ['centroid', 'similarity', 'max_iters']
        assert criterion in criteria
        if criterion == 'max_iters':
        #dừng khi số lần lặp vượt qua giới hạn max_iters
            if self._iteration >= threshold:
                return True
            else:
                return False
        elif criterion == 'centroid':
        #dừng khi số lượng centroid thay đổi không đáng kể
            E_new = [list(cluster._centroid) for cluster in self._cluster]
            E_new_minus_E = [centroid for centroid in E_new if centroid not in self._E]
            self._E = E_new
            if len(E_new_minus_E) <= threshold:
                return True
            else:
                return False
        else:
        #dừng khi độ giảm lỗi phân cụm ít hơn ngưỡng
            new_S_minus_S= self._new_S - self._S
            self._S= self._new_S
            if new_S_minus_S <= threshold:
                return True
            else:
                return False
        
    def run(self, seed_value, criterion, threshold):
        self.random_init(seed_value)
        
        #lặp cho tới điều kiện dừng ()
        self._iteration= 0
        while True:
            #reset các cluster, chỉ giữ lại các centroid
            for cluster in self._cluster:
                cluster.reset_members()
            self._new_S= 0
            for member in self._data:
                max_S= self.select_cluster_for(member)
                self._new_S += max_S
            for cluster in self._cluster:
                self.update_centroid_of(cluster)
            self._iteration += 1
            if self.stopping_condition(criterion, threshold):
                break
        print ('Done')
                
    def compute_purity(self):
    #đánh giá chất lượng phân cụm bằng cách tính purity
        majority_sum= 0
        for cluster in self._cluster:
            member_labels = [member._label for member in cluster._members]
            max_count = max([member_labels.count(label) for label in range(20)])
            majority_sum += max_count
        return majority_sum * 1. / len(self._data)
    
    def compute_NMI(self):
    #đánh giá chất lượng phân cụm bằng cách tính NMI
        I_value, H_omega, H_C, N= 0., 0., 0., len(self._data)
        for cluster in self._cluster:
            wk= len(cluster._members) * 1. #change int to real
            H_omega += - wk / N * np.log10(wk / N)
            member_labels = [member._label for member in cluster._members]
        for label in range(20):
            wk_cj= member_labels.count(label) * 1.
            cj= self._label_count[label]
            I_value += wk_cj /N * np.log10(N* wk_cj /(wk * cj) + 1e-12) #tránh err khi log số quá nhỏ
        for label in range(20):
            cj = self._label_count[label] * 1.
            H_C += - cj / N * np.log10(cj / N)
        return I_value * 2. /(H_omega + H_C) 
        

In [5]:
if __name__ == "__main__":
    max_Kmeans = -1

    #thử 5 lần, chọn kết quả tốt nhất
    for _ in range(5):
        kmeans = Kmeans(20)
        kmeans.load_data('./datasets/20news-bydate/words_tf_idf.txt')
        kmeans.run(seed_value=20, criterion='similarity', threshold=1e5)
        compute_NMI = kmeans.compute_NMI()
        if (max_Kmeans < compute_NMI):
            max_Kmeans = compute_NMI
            result = kmeans
    print(kmeans.compute_purity())

Done


MemoryError: Unable to allocate 29.9 MiB for an array with shape (386, 10150) and data type float64